In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import re


In [2]:
df = pd.read_csv('../../datasets/telegram/fakeTelegram.BR_2022.csv')
df.head()

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,5,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Texto,telegram,NaN,NaN
1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",5,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Texto,telegram,NaN,NaN
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,"É isso, nossa parte já foi quase toda feita. N...",5,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Texto,telegram,NaN,NaN
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,5,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,NaN,True,False,False,NaN,5,2022-10-05 06:28:29.316325,NaN,NaN,507185,Imagem,telegram,NaN,5ad5c8bbe9da93a37fecf3e5aa5b0637


In [3]:
df.shape

(557586, 19)

In [4]:

df = df[df['trava_zap'] == False]
df.shape

(557570, 19)

In [5]:

# eliminando trava_zap
df['text_content_anonymous'] = df['text_content_anonymous'].astype("string")
df['caracteres_especial_count'] = df['text_content_anonymous'].apply(
    lambda x: len(re.findall(r'[^A-Za-z0-9]', str(x))) if pd.notnull(x) else 0
)

num_caracteres_elevado = df['caracteres_especial_count'] >= 1900
df_final = df[~num_caracteres_elevado]
df_final.shape

(557515, 20)

In [6]:
table = pa.Table.from_pandas(df_final)

In [7]:
parquet_file_path = "../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet"
pq.write_table(table, parquet_file_path)


In [2]:
import duckdb

duckdb.sql("SELECT * FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet'") 

┌─────────────────────┬──────────────────────┬───┬──────────────────────┬──────────────────────┬───────────────────┐
│    date_message     │ id_member_anonymous  │ … │      media_md5       │ caracteres_especia…  │ __index_level_0__ │
│       varchar       │       varchar        │   │       varchar        │        int64         │       int64       │
├─────────────────────┼──────────────────────┼───┼──────────────────────┼──────────────────────┼───────────────────┤
│ 2022-10-05 06:25:04 │ 1078cc958f0febe28f…  │ … │ NULL                 │                   24 │                 0 │
│ 2022-10-05 06:25:08 │ NULL                 │ … │ NULL                 │                   29 │                 1 │
│ 2022-10-05 06:26:28 │ 92a2d8fd7144074f65…  │ … │ NULL                 │                   77 │                 2 │
│ 2022-10-05 06:27:28 │ d60aa38f62b4977426…  │ … │ 94dca4cda503100ebf…  │                    8 │                 3 │
│ 2022-10-05 06:27:44 │ cd6979b0b5265f0846…  │ … │ 5ad5c8bbe9da9

In [12]:
# quantidade de registros
duckdb.sql("SELECT count(*) FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet' ")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       557515 │
└──────────────┘

In [21]:
result = duckdb.sql("SELECT * FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet'") 
print(result.columns)

['date_message', 'id_member_anonymous', 'id_group_anonymous', 'media', 'media_type', 'media_url', 'has_media', 'has_media_url', 'trava_zap', 'text_content_anonymous', 'dataset_info_id', 'date_system', 'score_sentiment', 'score_misinformation', 'id_message', 'message_type', 'messenger', 'media_name', 'media_md5', 'caracteres_especial_count', '__index_level_0__']


In [42]:
# quantidade de mensagens não vazias
duckdb.sql("SELECT COUNT(text_content_anonymous) AS count_messages FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet' ")

┌────────────────┐
│ count_messages │
│     int64      │
├────────────────┤
│         444146 │
└────────────────┘

In [41]:
# quantidade de usuários distintos
duckdb.sql("SELECT COUNT(DISTINCT(id_member_anonymous)) AS count_id_member" \
" FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet' ")

┌─────────────────┐
│ count_id_member │
│      int64      │
├─────────────────┤
│           14808 │
└─────────────────┘

In [ ]:
# quantidade de grupos distintos
duckdb.sql("SELECT COUNT(DISTINCT(id_group_anonymous)) AS count_id_group " \
" FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet' ")

┌────────────────┐
│ count_id_group │
│     int64      │
├────────────────┤
│            178 │
└────────────────┘

In [ ]:
duckdb.sql("SELECT COUNT(*) AS count_only_text FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet'" \
" WHERE REGEXP_MATCHES(text_content_anonymous, '^[a-zA-Z ]*$')")

┌─────────────────┐
│ count_only_text │
│      int64      │
├─────────────────┤
│           10091 │
└─────────────────┘

In [29]:
duckdb.sql("SELECT has_media FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet'")

┌────────────────────────┐
│       has_media        │
│        boolean         │
├────────────────────────┤
│ false                  │
│ false                  │
│ false                  │
│ true                   │
│ true                   │
│ false                  │
│ false                  │
│ true                   │
│ true                   │
│ false                  │
│   ·                    │
│   ·                    │
│   ·                    │
│ false                  │
│ true                   │
│ false                  │
│ false                  │
│ false                  │
│ false                  │
│ true                   │
│ true                   │
│ false                  │
│ true                   │
├────────────────────────┤
│         ? rows         │
│ (>9999 rows, 20 shown) │
└────────────────────────┘

In [38]:
# quantidade de mensagens que contém midia
duckdb.sql("SELECT COUNT(*) AS count_true_media FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet' " \
"WHERE has_media = true")

┌──────────────────┐
│ count_true_media │
│      int64       │
├──────────────────┤
│           332604 │
└──────────────────┘

In [34]:

duckdb.sql("SELECT media_type FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet' ")

┌────────────────────────┐
│       media_type       │
│        varchar         │
├────────────────────────┤
│ NULL                   │
│ NULL                   │
│ NULL                   │
│ image/jpg              │
│ image/jpg              │
│ NULL                   │
│ NULL                   │
│ image/jpg              │
│ image/jpg              │
│ NULL                   │
│  ·                     │
│  ·                     │
│  ·                     │
│ NULL                   │
│ video/mp4              │
│ NULL                   │
│ NULL                   │
│ NULL                   │
│ NULL                   │
│ image/jpg              │
│ image/jpg              │
│ NULL                   │
│ url                    │
├────────────────────────┤
│         ? rows         │
│ (>9999 rows, 20 shown) │
└────────────────────────┘

In [ ]:
query = """
SELECT
    SUM(CASE WHEN media_type = 'url' THEN 1 ELSE 0 END) AS count_url,
    SUM(CASE WHEN media_type = 'video/mp4' THEN 1 ELSE 0 END) AS count_video,
    SUM(CASE WHEN media_type = 'image/jpg' THEN 1 ELSE 0 END) AS count_jpg
FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet'
"""

result = duckdb.sql(query).df()
print(result)


   count_url  count_video  count_jpg
0   100856.0      18497.0   200440.0


In [37]:
duckdb.sql("SELECT COUNT(*) AS count_jpg FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet' " \
"WHERE media_type = 'image/jpg' ")

┌───────────┐
│ count_jpg │
│   int64   │
├───────────┤
│    200440 │
└───────────┘

In [43]:
duckdb.sql("SELECT COUNT(*) AS count_video FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet' " \
"WHERE media_type = 'video/mp4' ")

┌─────────────┐
│ count_video │
│    int64    │
├─────────────┤
│       18497 │
└─────────────┘

In [44]:
duckdb.sql("SELECT COUNT(*) AS count_url FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet' " \
"WHERE media_type = 'url' ")

┌───────────┐
│ count_url │
│   int64   │
├───────────┤
│    100856 │
└───────────┘

In [47]:
duckdb.sql("SELECT media_url, COUNT(media_url) AS count_url FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet'" \
"GROUP BY media_url " \
"ORDER BY count_url DESC LIMIT 30 ")

┌────────────────────────────────────────────────────────────────────────────────────┬───────────┐
│                                     media_url                                      │ count_url │
│                                      varchar                                       │   int64   │
├────────────────────────────────────────────────────────────────────────────────────┼───────────┤
│ t.me/alexeconomia                                                                  │      4159 │
│ https://youtube.com/c/especulandoosfatosoficial                                    │      1607 │
│ t.me/fimtaproximo                                                                  │      1581 │
│ t.me/+EWlGMatRZGg3OTlh                                                             │      1419 │
│ https://youtu.be/qbTzhB0akt8                                                       │      1160 │
│ https://youtu.be/zDuOoyhyN-4                                                       │      1022 │
│ t.me/+ew

In [30]:
duckdb.sql("SELECT REGEXP_EXTRACT(media_url, '^(?:https?://)?(?:www\.)?([^/]+)', 1) AS dominio, COUNT(*) AS count_dominio FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet'" \
"WHERE media_url IS NOT NULL" \
" GROUP BY dominio ORDER BY COUNT(*) DESC LIMIT 30")

┌─────────────────────────────┬───────────────┐
│           dominio           │ count_dominio │
│           varchar           │     int64     │
├─────────────────────────────┼───────────────┤
│ youtu.be                    │         52842 │
│ t.me                        │         20323 │
│ youtube.com                 │         14768 │
│ terrabrasilnoticias.com     │          8284 │
│ instagram.com               │          6626 │
│ jornaldacidadeonline.com.br │          4829 │
│ twitter.com                 │          4441 │
│ gazetabrasil.com.br         │          1781 │
│ pleno.news                  │          1602 │
│ facebook.com                │          1368 │
│      ·                      │            ·  │
│      ·                      │            ·  │
│      ·                      │            ·  │
│ m.youtube.com               │           694 │
│ noticias.uol.com.br         │           629 │
│ tribunanacional.com.br      │           609 │
│ fb.watch                    │         

In [ ]:
# Retornando os 30 usuários mais ativos
duckdb.sql("SELECT id_member_anonymous, COUNT(id_member_anonymous) AS count_member FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet'" \
"GROUP BY id_member_anonymous " \
"ORDER BY count_member DESC LIMIT 30 ")

┌──────────────────────────────────┬──────────────┐
│       id_member_anonymous        │ count_member │
│             varchar              │    int64     │
├──────────────────────────────────┼──────────────┤
│ abe534d581ec6d552243d6955d3c3cd8 │        12289 │
│ 1665e22b0f564cd46d343f7677014821 │         5452 │
│ 1ac091b8ed5c4e42383f1b4ff4cc9b2d │         5060 │
│ c743967449a387ad2c1c7e03b2c45b36 │         3019 │
│ e7998863ac2a40086657fab4a6b463c9 │         1928 │
│ e8fd8fee8c39342d37993775da7756d5 │         1706 │
│ 2f4be6244ede15b46e8329a2c975be30 │         1620 │
│ d49f81df0c75d1d72bee6c5b2d707da0 │         1571 │
│ 4f7d493f0f6222d56e5b19a4f7c336cc │         1447 │
│ 773b9bd5b02a2e96f9d732c29bfcb663 │         1411 │
│                ·                 │           ·  │
│                ·                 │           ·  │
│                ·                 │           ·  │
│ beb77036ff26e78865da19310ac95169 │          897 │
│ 50cc2b20a3c2c24ce68bb2c0554ddae1 │          852 │
│ fbbbfb737a

In [22]:
# Retornando os 30 usuários que mais compartilham mensagem de texto
duckdb.sql("SELECT id_member_anonymous, " \
"COUNT(*) as count_text FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet' " \
"WHERE REGEXP_MATCHES(text_content_anonymous, '^[a-zA-Z ]*$') AND  id_member_anonymous IS NOT NULL  " \
"GROUP BY id_member_anonymous ORDER BY count_text DESC LIMIT 30")

┌──────────────────────────────────┬────────────┐
│       id_member_anonymous        │ count_text │
│             varchar              │   int64    │
├──────────────────────────────────┼────────────┤
│ e7998863ac2a40086657fab4a6b463c9 │        189 │
│ e5e3e38d433016adf62ea34432853a3c │        145 │
│ b931bad9b759d8f473fc8eb65ed22426 │         61 │
│ e8fd8fee8c39342d37993775da7756d5 │         54 │
│ 09605fc0dabc0212d5566b001e4fe94d │         37 │
│ 065039508ed6e276e5f092dd22eaef94 │         36 │
│ 762eded3b216b0a586c9488078c3574a │         30 │
│ c052c859b42c5a1923c22f5a201de746 │         26 │
│ a3dbfcf3ede2f8912d7c6b85f0966586 │         25 │
│ bd7859770eef94f34d0344a5397c5443 │         24 │
│                ·                 │          · │
│                ·                 │          · │
│                ·                 │          · │
│ 26b496125c8b6773bce453e14f172430 │         19 │
│ 01642d1a0ebc95e3be57710a9e077dd7 │         18 │
│ 79fa510c3ab44548d903284ab9162c9d │         18 │


In [21]:
# Os 30 usuários que mais compartilharam mídias; 

duckdb.sql("" \
"SELECT id_member_anonymous, COUNT(*) as count_midia FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet' " \
"WHERE has_media = true AND id_member_anonymous IS NOT NULL GROUP BY id_member_anonymous ORDER BY count_midia DESC LIMIT 30")


┌──────────────────────────────────┬─────────────┐
│       id_member_anonymous        │ count_midia │
│             varchar              │    int64    │
├──────────────────────────────────┼─────────────┤
│ 1ac091b8ed5c4e42383f1b4ff4cc9b2d │        4632 │
│ 1665e22b0f564cd46d343f7677014821 │        2972 │
│ e7998863ac2a40086657fab4a6b463c9 │        1498 │
│ 2f4be6244ede15b46e8329a2c975be30 │        1489 │
│ 4f7d493f0f6222d56e5b19a4f7c336cc │        1359 │
│ c052c859b42c5a1923c22f5a201de746 │        1289 │
│ 773b9bd5b02a2e96f9d732c29bfcb663 │        1167 │
│ 56b8359fd127312651b80b8ed8030085 │        1079 │
│ e8fd8fee8c39342d37993775da7756d5 │        1007 │
│ 3e49fd40fd973ee1b8f1a6d58feb4a54 │         965 │
│                ·                 │          ·  │
│                ·                 │          ·  │
│                ·                 │          ·  │
│ f4561a21fb26cfbebf48dcd5c557f82f │         650 │
│ aed9b55fd26f99a58a11c11fa2af80ef │         644 │
│ cb68fd028cfdfc837248144a18673

In [ ]:
# As 30 mensagens mais compartilhadas; 
duckdb.sql("SELECT text_content_anonymous, COUNT(text_content_anonymous) AS count_text FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet'" \
"GROUP BY text_content_anonymous " \
"ORDER BY count_text DESC LIMIT 30 ")

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┐
│                                         text_content_anonymous                                          │ count_text │
│                                                 varchar                                                 │   int64    │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────┼────────────┤
│ This community was blocked in Brazil following a decision of the Superior Electoral Court (TSE).        │      17422 │
│ Rough_sex🙈                                                                                             │       1134 │
│ Anal sex🙈                                                                                              │       1118 │
│ سکس مردان ازبک با زن انگلیسی با این vpn از سایتهای ممنوعه ببینید🙈\n\nشاهد الرجال الأوزبكيين يمارسون …  │       1019 │
│ https://youtu.be/qbTzhB0akt8     

In [17]:
# As 30 mensagens mais compartilhadas em grupos diferentes; 
duckdb.sql("SELECT text_content_anonymous, COUNT( DISTINCT(id_group_anonymous)) AS disticount_group" 
" FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet'" \
"WHERE text_content_anonymous IS NOT NULL " \
"GROUP BY text_content_anonymous " \
"ORDER BY disticount_group DESC LIMIT 30 ")

┌───────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────┐
│                                      text_content_anonymous                                       │ disticount_group │
│                                              varchar                                              │      int64       │
├───────────────────────────────────────────────────────────────────────────────────────────────────┼──────────────────┤
│ This community was blocked in Brazil following a decision of the Superior Electoral Court (TSE).  │               59 │
│ BOA NOITE A TODOS. O QUE EU VOU DIZER É SÉRIO A ESQUERDA ESTA AGINDO NOS GRUPOS DE WHATSAPP POS…  │               36 │
│ Hoje às 18 horas no canal Inteligência Ltda do YouTube \nhttps://youtube.com/c/Intelig%C3%AAnci…  │               35 │
│ [USER]                                                                                            │               32 │
│ https://youtu.be/zDuOoyhyN-4  

In [ ]:
# Mensagens idênticas compartilhadas pelo mesmo usuário (e suas quantidades)
duckdb.sql(
    "SELECT text_content_anonymous, id_member_anonymous, count(*) as quantidade FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet' " \
    "WHERE text_content_anonymous IS NOT NULL AND id_member_anonymous IS NOT NULL "\
    "GROUP BY text_content_anonymous, id_member_anonymous " \
    "HAVING count(*) > 1 ORDER BY quantidade DESC ")

┌──────────────────────────────────────────────────────────────────────┬──────────────────────────────────┬────────────┐
│                        text_content_anonymous                        │       id_member_anonymous        │ quantidade │
│                               varchar                                │             varchar              │   int64    │
├──────────────────────────────────────────────────────────────────────┼──────────────────────────────────┼────────────┤
│ This community was blocked in Brazil following a decision of the S…  │ f233cf8b1d4ede06f32199a7e0081bf5 │        838 │
│ This community was blocked in Brazil following a decision of the S…  │ c743967449a387ad2c1c7e03b2c45b36 │        532 │
│ This community was blocked in Brazil following a decision of the S…  │ 7696d5103cdb8ac352d748a1db1126b0 │        530 │
│ Bem vindo(a) ao grupo Ipirá Notícias. \n\nCompartilhe nosso link. …  │ 1665e22b0f564cd46d343f7677014821 │        357 │
│ This community was blocked in 

In [27]:
# Mensagens idênticas compartilhadas pelo mesmo usuário em grupos distintos (e suas quantidades);
duckdb.sql(
    "SELECT " \
    "text_content_anonymous, " \
    "id_member_anonymous, " \
    "count(*) as quantidade," \
    "COUNT( DISTINCT(id_group_anonymous)) AS disticount_group_count " \
    " FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet' " \
    "WHERE text_content_anonymous IS NOT NULL AND id_member_anonymous IS NOT NULL "\
    "GROUP BY text_content_anonymous, id_member_anonymous " \
    "HAVING count(*) > 1 ORDER BY quantidade, disticount_group_count DESC ")

┌─────────────────────────────────────────────┬──────────────────────────────────┬────────────┬────────────────────────┐
│           text_content_anonymous            │       id_member_anonymous        │ quantidade │ disticount_group_count │
│                   varchar                   │             varchar              │   int64    │         int64          │
├─────────────────────────────────────────────┼──────────────────────────────────┼────────────┼────────────────────────┤
│ - Contra tudo e contra todos, tivemos no …  │ 32783be6c9de86a5b268719d4a151a93 │          2 │                      2 │
│ DAS DUAS UMA, OU METERAM A MÃO NA CARA DU…  │ 8e72a8fbc76263914ed485ab068396fd │          2 │                      2 │
│ https://vm.tiktok.com/ZMFFdHemm             │ 1d0f50675e9ae0d02d348ee539945ddf │          2 │                      2 │
│ https://t.me/+hXARlf2UnEEwOTVh\nESTÃO QUE…  │ 27cd6763c7e92cf24461024c99369d0e │          2 │                      2 │
│ 🚨🚨 No primeiro turno fui mesár

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


consulta_limpa = duckdb.sql("SELECT LOWER(text_content_anonymous) AS texto FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet'" \
" WHERE text_content_anonymous IS NOT NULL").df()

vectorizer = CountVectorizer(ngram_range=(1, 3), max_features=30)
X = vectorizer.fit_transform(consulta_limpa["texto"])


ngrams = vectorizer.get_feature_names_out()
counts = X.toarray().sum(axis=0)
result = sorted(zip(ngrams, counts), key=lambda x: x[1], reverse=True)


In [5]:
print(result)

[('de', 474760), ('que', 306455), ('do', 207160), ('com', 188125), ('https', 168455), ('não', 164519), ('para', 163892), ('no', 131670), ('em', 130060), ('da', 127004), ('se', 111695), ('os', 93381), ('um', 82615), ('bolsonaro', 78953), ('na', 76359), ('user', 74344), ('as', 69542), ('por', 67090), ('uma', 62875), ('be', 61236), ('mais', 59149), ('youtu', 58208), ('youtu be', 58207), ('https youtu', 57882), ('https youtu be', 57882), ('me', 54837), ('lula', 53711), ('brasil', 53117), ('ao', 48230), ('como', 45471)]


In [6]:
duckdb.sql("SELECT score_sentiment FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet' ")

┌────────────────────────┐
│    score_sentiment     │
│         double         │
├────────────────────────┤
│                    0.0 │
│                 0.0644 │
│                -0.3551 │
│                    0.0 │
│                   NULL │
│                 0.7003 │
│                 0.9716 │
│                   NULL │
│                   NULL │
│                -0.8779 │
│                     ·  │
│                     ·  │
│                     ·  │
│                    0.0 │
│                    0.0 │
│                    0.0 │
│                -0.6249 │
│                    0.0 │
│                 -0.765 │
│                   NULL │
│                   NULL │
│                -0.7269 │
│                    0.0 │
├────────────────────────┤
│         ? rows         │
│ (>9999 rows, 20 shown) │
└────────────────────────┘

In [9]:
# As 30 mensagens mais positivas (distintas)
duckdb.sql("SELECT DISTINCT(text_content_anonymous), score_sentiment" \
" FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet' " \
"WHERE  score_sentiment >= 0.05 " \
"ORDER BY score_sentiment DESC LIMIT 30")

┌────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────┐
│                                       text_content_anonymous                                       │ score_sentiment │
│                                              varchar                                               │     double      │
├────────────────────────────────────────────────────────────────────────────────────────────────────┼─────────────────┤
│ Shemoneh Esreh – Amidá – A Grande Oração\n\n ,Avot - Benção dos Patriarcas\n\nÓ Senhor, abre os …  │          0.9992 │
│ MENTIROSA!!!\n\n\nPensa direita, não é correto compartilhar um vídeo manipulado, sem fonte e que…  │           0.999 │
│ Pensa direita, não é correto compartilhar um vídeo manipulado, sem fonte e que deturpa fatos rea…  │           0.999 │
│ בונה ירושלים, Bo'ne Yetrushalayim - Benção da Reconstrução de Jerusalém\n\nE retorna com miseric…  │          0.9989 │
│ Os Sete Samurais \n \nQuando f

In [10]:
# As 30 mensagens mais negativas (distintas)
duckdb.sql("SELECT DISTINCT(text_content_anonymous), score_sentiment" \
" FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet' " \
"WHERE  score_sentiment <= 0.05 " \
"ORDER BY score_sentiment DESC LIMIT 30")

┌────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────┐
│                                       text_content_anonymous                                       │ score_sentiment │
│                                              varchar                                               │     double      │
├────────────────────────────────────────────────────────────────────────────────────────────────────┼─────────────────┤
│ Carluxo internado numa sexta a noite? Algo não tá cheirando bem. 😂                                │          0.0498 │
│ Se vc se acha tão bem informado saia desse grupo que não te agrega em nada.                        │          0.0498 │
│ O Presidente já falou alguma coisa  ? Com certeza esta organizando algo para nós direcionar        │          0.0498 │
│ Esse vídeo é demais. Mas não consigo enviar no zap. Só vai 2min50seg. Me ajuda. Como faço.         │          0.0471 │
│ Deus tem que dar uma luz pra ge

In [20]:
# O usuário mais otimista; 
duckdb.sql("SELECT id_member_anonymous, AVG(score_sentiment) AS media_score " \
" FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet' " \
"GROUP BY id_member_anonymous " \
"ORDER BY media_score DESC LIMIT 1")

┌──────────────────────────────────┬─────────────┐
│       id_member_anonymous        │ media_score │
│             varchar              │   double    │
├──────────────────────────────────┼─────────────┤
│ 016723c9eb650754d3dfe9b3ba69e5c2 │      0.9953 │
└──────────────────────────────────┴─────────────┘

In [21]:
# O usuário mais pessimista; 
duckdb.sql("SELECT id_member_anonymous, AVG(score_sentiment) AS media_score " \
" FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet' " \
"GROUP BY id_member_anonymous " \
"ORDER BY media_score LIMIT 1")

┌──────────────────────────────────┬─────────────┐
│       id_member_anonymous        │ media_score │
│             varchar              │   double    │
├──────────────────────────────────┼─────────────┤
│ 243a43d802953032e1ea9a363546c3b8 │     -0.9992 │
└──────────────────────────────────┴─────────────┘

In [22]:
# As 30 maiores mensagens; 
duckdb.sql("SELECT text_content_anonymous, length(text_content_anonymous) AS numero_string " \
"FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet' " \
"ORDER BY numero_string DESC LIMIT 30")

┌──────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────┐
│                                        text_content_anonymous                                        │ numero_string │
│                                               varchar                                                │     int64     │
├──────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────────┤
│ CCCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHOOOOOOOOOOOOOOOOOOOOOORRRRRRRRRRRRRRRRRRRRRRRRAAAAAAAA…  │          4096 │
│ 1: Sansão foi a Gaza, e viu ali uma prostituta, e entrou a ela. \n2: E foi dito aos gazitas: Sansã…  │          4096 │
│ CCCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHOOOOOOOOOOOOOOOOOOOOOORRRRRRRRRRRRRRRRRRRRRRRRAAAAAAAA…  │          4096 │
│ CCCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHOOOOOOOOOOOOOOOOOOOOOORRRRRRRRRRRRRRRRRRRRRRRRAAAAAAAA…  │          4096 │
│ 1: Aconteceu também naqueles d

In [23]:
# As 30 menores mensagens; 
duckdb.sql("SELECT text_content_anonymous, length(text_content_anonymous) AS numero_string " \
"FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet' " \
"ORDER BY numero_string LIMIT 30")

┌────────────────────────┬───────────────┐
│ text_content_anonymous │ numero_string │
│        varchar         │     int64     │
├────────────────────────┼───────────────┤
│ 🤯                     │             1 │
│ 😎                     │             1 │
│ 😢                     │             1 │
│ 💣                     │             1 │
│ 🤔                     │             1 │
│ 🌩                      │             1 │
│ Q                      │             1 │
│ 😅                     │             1 │
│ 🌹                     │             1 │
│ 🤭                     │             1 │
│ ·                      │             · │
│ ·                      │             · │
│ ·                      │             · │
│ .                      │             1 │
│ .                      │             1 │
│ .                      │             1 │
│ 🔥                     │             1 │
│ 🌩                      │             1 │
│ 🤭                     │             1 │
│ 👀                  

In [25]:
# O dia em que foi publicado a maior quantidade de mensagens; 
duckdb.sql("SELECT date_message, length(text_content_anonymous) AS numero_string " \
"FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet' " \
"ORDER BY numero_string DESC LIMIT 1")

┌─────────────────────┬───────────────┐
│    date_message     │ numero_string │
│       varchar       │     int64     │
├─────────────────────┼───────────────┤
│ 2022-10-05 08:02:17 │          4096 │
└─────────────────────┴───────────────┘

In [28]:
# As mensagens que possuem as palavras “FACÇÃO” e “CRIMINOSA”; 
duckdb.sql("SELECT text_content_anonymous FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet'" \
"WHERE text_content_anonymous LIKE '%FACÇÃO%' AND text_content_anonymous LIKE '%CRIMINOSA%' ")

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                text_content_anonymous                                                │
│                                                       varchar                                                        │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ #lulaladrãoseulugarénaprisão \n#PTFACÇÃOCRIMINOSA \n\n\n\nhttps://gettr.com/post/p1t8lax7bb8                         │
│ PT NÃO É PARTIDO POLÍTICO É FACÇÃO CRIMINOSA ... BANDIDOS SUJOS                                                      │
│ 🚨 LULA USA BONE COM SIGLA CPX NO COMPLEXO DO ALEMAO!.🚨\n\n🚨 CPX  É UMA FACÇÃO CRIMINOSA DO ALEMAO ALIADA AO CV …  │
│ 🚨 LULA USA BONE COM SIGLA CPX NO COMPLEXO DO ALEMAO!.🚨\n\n🚨 CPX  É UMA FACÇÃO CRIMINOSA DO ALEMAO ALIADA AO CV …  │
│ 🚨 LULA USA BONE COM SIGLA CPX NO COM

In [29]:
# As mensagens que possuem a palavra “SEGURANÇA”. 
duckdb.sql("SELECT text_content_anonymous FROM '../datasets/fakeTelegram_SemTravaZap_BR_2022.parquet'" \
"WHERE text_content_anonymous LIKE '%SEGURANÇA%'  ")

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                text_content_anonymous                                                │
│                                                       varchar                                                        │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Espalhe!! Divulgue!! EXIJA!!\nNão entre nesse jogo de composição do Congresso e Governadores que apoiam Bolsonaro.…  │
│ ATENÇÃO A TODAS E SOMENTE MULHERES DESTE GRUPO\n\nPOR SABER QUE VCS SÃO A MAIORIA EM TODO MUNDO E SABER DO PODER Q…  │
│ Espalhe!! Divulgue!! EXIJA!!\nNão entre nesse jogo de composição do Congresso e Governadores que apoiam Bolsonaro.…  │
│ Bolsonaro ou Lula?\nO que cada qual já fez e propõe para a *SEGURANÇA PÚBLICA*:\n\nO presidente Bolsonaro consegui…  │
│ *VIDEO: FUNCIONARIO DA ONU ADM